In [2]:
# === 1. Install dependencies ===
!pip install facenet-pytorch transformers scikit-learn tqdm

  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pillow-10.2.0-cp310-cp310-win_amd64.whl.metadata (9.9 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.2.2-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached torchvision-0.17.2-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached charset_normalizer-3.4.3-cp310-cp310-win_amd64.whl.metadata (37 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2025.7.34-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cac

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\EliteLaptop\\AppData\\Local\\Temp\\pip-unpack-pl_ywp2a\\torch-2.2.2-cp310-cp310-win_amd64.whl'
Consider using the `--user` option or check the permissions.



In [1]:
# T-CLiE_pipeline_extraction_baseline.ipynb

# === 1. Install dependencies ===
!pip install facenet-pytorch transformers scikit-learn tqdm

# === 2. Imports & utils ===
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN
from transformers import ViTFeatureExtractor, ViTModel, CLIPProcessor, CLIPModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === 3. Load pretrained models ===

# e4e encoder (StyleGAN2 inversion)
# Repo: https://github.com/omertov/encoder4editing
from encoder4editing.model.psp import pSp
import sys
sys.path.append('encoder4editing')  # adapte selon ton arborescence

e4e_ckpt_path = 'encoder4editing/pretrained_models/e4e_ffhq_encode.pt'
e4e_model = pSp().to(device)
ckpt = torch.load(e4e_ckpt_path, map_location=device)
e4e_model.load_state_dict(ckpt['encoder'], strict=False)
e4e_model.eval()

# ViT model for patch features
vit_feat_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k').to(device).eval()

# CLIP model
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device).eval()

# === 4. Face cropper ===
mtcnn = MTCNN(image_size=224, margin=10, device=device)

# === 5. Video frame extractor & preprocessing ===
def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    frames = []
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = np.linspace(0, length-1, num_frames).astype(int)
    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    cap.release()
    return frames

def preprocess_frame(frame):
    # Face crop + resize 224x224
    face = mtcnn(frame)
    if face is None:
        return None
    return face.unsqueeze(0).to(device)  # [1,3,224,224]

# === 6. Feature extraction functions ===
@torch.no_grad()
def extract_z_t(face_tensor):
    # e4e expects normalized input [-1,1]
    x = (face_tensor - 0.5) * 2
    latent = e4e_model.encoder(x)[0]  # [1,512]
    return latent.squeeze(0).cpu().numpy()

@torch.no_grad()
def extract_v_t(face_tensor):
    inputs = vit_feat_extractor(images=face_tensor.squeeze(0).permute(1,2,0).cpu().numpy(), return_tensors="pt")
    inputs = {k: v.to(device) for k,v in inputs.items()}
    outputs = vit_model(**inputs)
    cls_emb = outputs.last_hidden_state[:,0,:]  # CLS token
    return cls_emb.squeeze(0).cpu().numpy()

@torch.no_grad()
def extract_c_t(face_tensor):
    inputs = clip_processor(images=face_tensor.squeeze(0).permute(1,2,0).cpu().numpy(), return_tensors="pt")
    inputs = {k: v.to(device) for k,v in inputs.items()}
    outputs = clip_model.get_image_features(**inputs)
    return outputs.squeeze(0).cpu().numpy()

# === 7. Extraction + caching pipeline ===
def process_video(video_path, cache_dir, prefix, num_frames=8):
    os.makedirs(cache_dir, exist_ok=True)
    basename = os.path.basename(video_path).split('.')[0]
    z_cache = os.path.join(cache_dir, f"{prefix}_{basename}_z.npy")
    v_cache = os.path.join(cache_dir, f"{prefix}_{basename}_v.npy")
    c_cache = os.path.join(cache_dir, f"{prefix}_{basename}_c.npy")

    if all(os.path.exists(p) for p in [z_cache, v_cache, c_cache]):
        z_seq = np.load(z_cache)
        v_seq = np.load(v_cache)
        c_seq = np.load(c_cache)
        return z_seq, v_seq, c_seq

    frames = extract_frames(video_path, num_frames)
    z_list, v_list, c_list = [], [], []
    for frame in frames:
        face = preprocess_frame(frame)
        if face is None:
            continue
        z = extract_z_t(face)
        v = extract_v_t(face)
        c = extract_c_t(face)
        z_list.append(z)
        v_list.append(v)
        c_list.append(c)

    if len(z_list) == 0:
        # fallback dummy
        z_seq = np.zeros((num_frames, 512))
        v_seq = np.zeros((num_frames, 768))
        c_seq = np.zeros((num_frames, 512))
    else:
        z_seq = np.vstack(z_list)
        v_seq = np.vstack(v_list)
        c_seq = np.vstack(c_list)

    np.save(z_cache, z_seq)
    np.save(v_cache, v_seq)
    np.save(c_cache, c_seq)
    return z_seq, v_seq, c_seq

# === 8. Δz computation ===
def compute_delta_z(z_seq):
    return np.diff(z_seq, axis=0)

# === 9. Dataset loading & feature extraction ===
def load_dataset(real_dir, fake_dir, cache_dir, num_frames=8):
    X, y = [], []
    real_videos = [os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.endswith('.mp4')]
    fake_videos = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir) if f.endswith('.mp4')]

    for video_path in tqdm(real_videos, desc="Real videos"):
        z_seq, v_seq, c_seq = process_video(video_path, cache_dir, 'real', num_frames)
        delta_z = compute_delta_z(z_seq)
        features = np.concatenate([
            np.mean(z_seq, axis=0),
            np.std(z_seq, axis=0),
            np.mean(v_seq, axis=0),
            np.mean(c_seq, axis=0),
            np.mean(delta_z, axis=0)
        ])
        X.append(features)
        y.append(0)

    for video_path in tqdm(fake_videos, desc="Fake videos"):
        z_seq, v_seq, c_seq = process_video(video_path, cache_dir, 'fake', num_frames)
        delta_z = compute_delta_z(z_seq)
        features = np.concatenate([
            np.mean(z_seq, axis=0),
            np.std(z_seq, axis=0),
            np.mean(v_seq, axis=0),
            np.mean(c_seq, axis=0),
            np.mean(delta_z, axis=0)
        ])
        X.append(features)
        y.append(1)

    return np.array(X), np.array(y)

# === 10. Train baseline classifier ===
def train_baseline(X, y):
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    clf = LogisticRegression(max_iter=500)
    clf.fit(Xs, y)
    preds = clf.predict_proba(Xs)[:,1]
    auc = roc_auc_score(y, preds)
    print(f"Baseline LogisticRegression AUC: {auc:.4f}")

# === 11. Main ===

if __name__ == '__main__':
    real_videos_dir = 'path_to_real_videos'
    fake_videos_dir = 'path_to_fake_videos'
    cache_dir = './feature_cache'
    num_frames = 8

    print("Extracting features and training baseline...")
    X, y = load_dataset(real_videos_dir, fake_videos_dir, cache_dir, num_frames)
    train_baseline(X, y)


^C


ModuleNotFoundError: No module named 'facenet_pytorch'

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\EliteLaptop\\AppData\\Local\\Temp\\pip-unpack-3yy21lpm\\transformers-4.55.2-py3-none-any.whl'
Consider using the `--user` option or check the permissions.



  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.9 MB 621.2 kB/s eta 0:00:03
   ---------------- ----------------------- 0.8/1.9 MB 817.9 kB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 817.9 kB/s eta 0:00:02
   ---------------------- ----------------- 1.0/1.9 MB 739.8 kB/s eta 0:00:02
   --------------------------- ------------ 1.3/1.9 MB 780.2 kB/s eta 0:00:01
   --------------------------------- ------ 1.6/1.9 MB 830.6 kB/s eta 0:00: